In [1]:
from google.cloud import bigquery
import sqlite3
import pandas as pd

In [2]:
client = bigquery.Client(project = 'wedge-project-0')
cursor = sqlite3.connect('sales.db')

In [3]:
sales_hour_query = client.query(
    """ 
    SELECT
  FORMAT_TIMESTAMP('%Y-%m-%d', datetime) AS date,
  FORMAT_TIMESTAMP('%H', datetime) AS hour,
  SUM(total) AS sales,
  
  COUNT(DISTINCT (
    CAST(FORMAT_TIMESTAMP('%Y-%m-%d', datetime) AS STRING) ||
    CAST(FORMAT_TIMESTAMP('%H', datetime) AS STRING) ||
    CAST(register_no AS STRING) ||
    CAST(emp_no AS STRING) ||
    CAST(trans_no AS STRING)
  )) AS unique_transactions,

  SUM(CASE
    WHEN trans_status IN ('V', 'R') THEN -1
    ELSE 1
  END) AS items

FROM
  `wedge-project-0.transactions.transArchive_*`
  
WHERE
  department != 0
  AND department != 15
  AND trans_status NOT IN ('M', 'C', 'J')


GROUP BY
  date, hour

ORDER BY
  date, hour

    """
) 

In [4]:
sales_owner_query = client.query(""" 
SELECT
  card_no,
  FORMAT_TIMESTAMP('%Y', datetime) AS year,
  FORMAT_TIMESTAMP('%m', datetime) AS month,
  SUM(total) AS sales,
  
  COUNT(DISTINCT (
    CAST(FORMAT_TIMESTAMP('%Y-%m-%d', datetime) AS STRING) ||
    CAST(FORMAT_TIMESTAMP('%H', datetime) AS STRING) ||
    CAST(register_no AS STRING) ||
    CAST(emp_no AS STRING) ||
    CAST(trans_no AS STRING)
  )) AS unique_transactions,

  SUM(CASE
    WHEN trans_status IN ('V', 'R') THEN -1
    ELSE 1
  END) AS items

FROM
  `wedge-project-0.transactions.transArchive_*`
  
WHERE
  department != 0
  AND department != 15
  AND trans_status NOT IN ('M', 'C', 'J')

GROUP BY
  year, month, card_no

ORDER BY
  year asc, month asc

""")


In [5]:
sales_description_query = client.query(
    """
    SELECT
  upc,description,department,
  FORMAT_TIMESTAMP('%Y', datetime) AS year,
  FORMAT_TIMESTAMP('%m', datetime) AS month,
  SUM(total) AS sales,
  
  COUNT(DISTINCT (
    CAST(FORMAT_TIMESTAMP('%Y-%m-%d', datetime) AS STRING) ||
    CAST(FORMAT_TIMESTAMP('%H', datetime) AS STRING) ||
    CAST(register_no AS STRING) ||
    CAST(emp_no AS STRING) ||
    CAST(trans_no AS STRING)
  )) AS unique_transactions,

  SUM(CASE
    WHEN trans_status IN ('V', 'R') THEN -1
    ELSE 1
  END) AS items

FROM
  `wedge-project-0.transactions.transArchive_*`
  
WHERE
  department != 0
  AND department != 15
  AND trans_status NOT IN ('M', 'C', 'J')

GROUP BY
  year, month,upc,description,department

ORDER BY
  year asc, month asc

    """)



In [6]:
sales_hour = sales_hour_query.to_dataframe()
sales_owner = sales_owner_query.to_dataframe()
sales_descrip = sales_description_query.to_dataframe() 

c:\Users\aport\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\aport\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
c:\Users\aport\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [7]:
def insert_table(dataframe, name):

    columns = dataframe.columns
    
    query = f'insert into {name} ('
    query += ', '.join(f'"{col}"'for col in columns) + ') values\n'
    
    value_strings = []
    for _,row in dataframe.iterrows():
        formatted_row = []
        for value in row:
            formatted_row.append(f"'{value}'") 
        value_strings.append(f"({', '.join(formatted_row)})")
    
    query += ', '.join(value_strings) + ';'
    


    cursor.execute(query)
    cursor.commit()

In [8]:
def create_table(dataframe,name):

    type_maps = {'int64':'INTEGER','float64':'FLOAT','object':'TEXT','datetime64[ns]':'TIMESTAMP'}

    columns = [(col,type_maps.get(str(types))) for col,types in dataframe.dtypes.items()]        



    query = f'create table if not exists {name}(\n'
    query += ',\n'.join(f'{col} {types}' for col, types in columns) + ')\n;'
    

    cursor.execute(query)
    cursor.commit()

    insert_table(dataframe,name)



In [9]:
# create_table(sales-hour,'sales_by_hour)
create_table(sales_owner,'sales_by_owner')
create_table(sales_descrip,'sales_by_description')